In [20]:
import numpy as np
from sentence_transformers import SentenceTransformer
import pandas as pd
from nltk.corpus import stopwords
import nltk
import re
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import FunctionTransformer
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\putte\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [2]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

c:\Users\putte\miniconda3\envs\dataviz\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [3]:
df = pd.read_csv("main_file.csv")

In [18]:
df_2018 = df.loc[df.year == 2018].dropna(subset=['abstract'])

In [22]:
def text_preprocessing(s):
    """
    - Lowercase the sentence
    - Change "'t" to "not"
    - Remove "@name"
    - Isolate and remove punctuations except "?"
    - Remove other special characters
    - Remove stop words except "not" and "can"
    - Remove trailing whitespace
    """
    s = s.lower()
    # Change 't to 'not'
    s = re.sub(r"\'t", " not", s)
    # Remove @name
    s = re.sub(r'(@.*?)[\s]', ' ', s)
    # Isolate and remove punctuations except '?'
    s = re.sub(r'([\'\"\!\?\\/\,])', r' \1 ', s)
    s = re.sub(r'[^\w\s\?\.\']', ' ', s)
    # Remove number
    s = re.sub(r'[0-9]', '', s)
    # Remove some special characters
    s = re.sub(r'([\;\:\|•«\n])', ' ', s)
    # Remove stopwords except 'not' and 'can'
    s = " ".join([word for word in s.split()
                  if word not in stopwords.words('english')
                  or word in ['not', 'can']])
    # Remove trailing whitespace
    s = re.sub(r'\s+', ' ', s).strip()

    return s

In [23]:
clean_cols = ['abstract']

txt_pipe = Pipeline([('clean_text', FunctionTransformer(lambda x: x.applymap(text_preprocessing)))])

col_trans = ColumnTransformer(transformers=[
    ('txt_pipe', txt_pipe, clean_cols),
    ], )

In [27]:
df_2018[clean_cols] = col_trans.fit_transform(df_2018)

C:\Users\putte\AppData\Local\Temp\ipykernel_22904\750581144.py:3: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  txt_pipe = Pipeline([('clean_text', FunctionTransformer(lambda x: x.applymap(text_preprocessing)))])


KeyboardInterrupt: 

In [ ]:
words = df_2018.abstract.values
vectors = model.encode(words)

In [ ]:
np.savetxt('2018_words.txt', words, fmt='%s')
np.save('2018_vectors.npy', vectors)